In [1]:
import pandas as pd
import numpy as np
import sys
from collections import deque

In [2]:
driverInfo = pd.read_csv('/Users/parthu/MyStuff/projects/Formula_One/Data Collection/driverInfo.csv')
constructorInfo = pd.read_csv('/Users/parthu/MyStuff/projects/Formula_One/Data Collection/constructorInfo.csv')
quali_results_I = pd.read_csv('/Users/parthu/MyStuff/projects/Formula_One/Data Collection/qualifying_results_I.csv')
quali_results_II = pd.read_csv('/Users/parthu/MyStuff/projects/Formula_One/Data Collection/qualifying_results_II.csv')
raceResults = pd.read_csv('/Users/parthu/MyStuff/projects/Formula_One/Data Collection/raceResults.csv')

In [3]:
raceDf = raceResults[['season','round','driverId','constructorId']]
unionDf = pd.merge(raceDf,raceDf,how="left",on=['season','round','constructorId'])
driverMatedf = unionDf.loc[unionDf['driverId_x'] != unionDf['driverId_y']].reset_index(drop=True)

In [4]:
driverMateAgg = driverMatedf.groupby(['constructorId','driverId_x','driverId_y'])['round'].count().reset_index()
driverMateAgg = driverMateAgg.loc[driverMateAgg['round'] > 3]
drivers = driverMateAgg.groupby(['driverId_x'])['driverId_y'].apply(list).reset_index(name='teammates').rename(columns={'driverId_x':'driverID'})

In [5]:
number_drivers = drivers.shape[0]
driver_network = np.zeros((number_drivers,number_drivers),dtype=int)
for i in range(0,number_drivers):
    for j in drivers['teammates'][i]:
        ind = drivers.loc[drivers['driverID'] == j].index
        driver_network[i][ind] = 1

In [7]:
def shortest_path(adj_matrix, start, end):
    num_nodes = adj_matrix.shape[0]
    
    # Initialize distance array
    distance = np.full(num_nodes, 1000, dtype=np.int64)
    distance[start] = 0
    
    # Initialize set of unvisited nodes
    unvisited = set(range(num_nodes))
    
    # Initialize previous node array to track the path
    previous = np.full(num_nodes, None)
    
    while unvisited:
        # Find the node with the minimum distance
        min_node = min(unvisited, key=lambda node: distance[node])
        
        # Remove the minimum node from unvisited set
        unvisited.remove(min_node)
        
        # Update distances for neighboring nodes
        for neighbor in range(num_nodes):
            if adj_matrix[min_node][neighbor] == 1:
                new_distance = distance[min_node] + 1
                if new_distance < distance[neighbor]:
                    distance[neighbor] = new_distance
                    previous[neighbor] = min_node
    
    # Build the shortest path
    path = deque()
    current = end
    while current is not None:
        path.appendleft(current)
        current = previous[current]
    
    return list(path)

In [8]:
adj_matrix = np.array([
    [0, 1, 1, 0, 0, 0],
    [1, 0, 1, 1, 0, 0],
    [1, 1, 0, 1, 1, 0],
    [0, 1, 1, 0, 1, 1],
    [0, 0, 1, 1, 0, 1],
    [0, 0, 0, 1, 1, 0]
])

start_node = 355
end_node = 166

shortest_path_nodes = shortest_path(driver_network, start_node, end_node)

print("Shortest path between nodes", start_node, "and", end_node, ":")
print(shortest_path_nodes)

Shortest path between nodes 355 and 166 :
[355, 39, 6, 386, 10, 166]


cleaning the drivers name from qualifying dataset 1. 

clean the drivers name for all special characters and spaces

In [10]:
new = quali_results_I['Car'].str.split(" ",expand=True)
quali_results_I['DriverName'] = quali_results_I['Driver'].str[:-4].str.replace(" ","").str.replace("-","").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
quali_results_I['DriverName'] = np.where((quali_results_I['DriverName'].str.contains('JacquesVilleneuve')) & (quali_results_I['season'] == 1983),quali_results_I['DriverName'].str.replace("JacquesVilleneuve","JacquesVilleneuveSr."),quali_results_I['DriverName'])
quali_results_I['Constructor'] = quali_results_I['Car'].str.split(" ",expand=True)[0]

In [12]:
driverInfo['DriverName'] = driverInfo['givenName'].str.replace(" ","").str.replace("-","").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')+driverInfo['familyName'].str.replace(" ","").str.replace("-","").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
join_df = pd.merge(quali_results_I,driverInfo,on=['season','DriverName'],how='left')[['Driver','Car','Time','season','round','DriverName','driverId','constructorId','Constructor']]
final_df = pd.merge(join_df,constructorInfo[['constructorId','name','season']],on=['season','constructorId'],how='left')
final_df['name'] = final_df['name'].astype(str)
final_df['Constructor'] = final_df['Constructor'].astype(str)
def check_partial_match(row):
    return ((row['Constructor'] in row['name']) |
            (('Larrousse' in row['name']) & ('Lola Ford' in row['Car']) & any(row['season'] == value for value in (1987,1988))) |
            (('Larrousse' in row['name']) & ('Lola Lamborghini' in row['Car']) & any(row['season'] == value for value in (1989,1990))) |
            (('Larrousse' in row['name']) & ('Venturi Lamborghini' in row['Car']) & (row['season'] == 1992))
           )

final_df['ConstructorMatch'] = final_df.apply(check_partial_match,axis=1)
true_df = final_df[final_df['ConstructorMatch'] == True]

In [13]:
quali_results_I.shape

(8505, 11)

In [14]:
true_df.shape

(8504, 11)